# Data voorbereiden trainen
We willen met databestanden 'tags' en 'ingredients' een classifier maken die voorspeld of het gerecht wel of niet uit dezelfde keuken komt als de anderen, op basis van maar een paar ingredienten. Kies hierbij maar 2 keukens, in dit geval 'nederlands' en 'italiaans' (dit zijn tags). op de kolommen zet je een paar veel voorkomende ingrediënten. Deze notebook laat een deel van het voorebereiden van de dat zien (door andere groepsgenoten verder uitgewerkt en generiek gemaakt).

### Imports

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
import random
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from IPython.display import display
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from userclass2 import User

### Data Inlezen

In [2]:
tagsFile = "tags"
ingredientsFile = "ingredients"

ingredientsdf = pd.read_csv(f"/data/foodboost/{ingredientsFile}.csv", index_col=0)
tagsdf = pd.read_csv(f"/data/foodboost/{tagsFile}.csv", index_col=0)
display(ingredientsdf)
display(tagsdf)

,recipe,ingredient,quantity,unit
0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,Kruidnoten met choco-discodip,discodip,2.0,el
3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g
...,...,...,...,...
71804,Biefstuk met rodewijnsaus en ham,olijfolie,4.0,el
71805,Biefstuk met rodewijnsaus en ham,biefstukken,4.0,NaN
71806,Biefstuk met rodewijnsaus en ham,boter,25.0,g
71807,Biefstuk met rodewijnsaus en ham,serranoham,4.0,plakken


,recipe,tag
0,Kruidnoten met choco-discodip,hollands
1,Kruidnoten met choco-discodip,gebak
2,Kruidnoten met choco-discodip,gebak
3,Kruidnoten met choco-discodip,sinterklaas
4,Kruidnoten met choco-discodip,sinterklaasavond
...,...,...
46035,Pittige truffels,nagerecht
46036,Biefstuk met rodewijnsaus en ham,snel
46037,Biefstuk met rodewijnsaus en ham,mediterraan
46038,Biefstuk met rodewijnsaus en ham,hoofdgerecht


### Basis functies
om zometeen makkelijk de ingredienten uit een recept te krijgen bijvoorbeeld

In [3]:
#INGREDIENTEN
# lijst met recepten of ingredienten ophalen
def recepten_bij_ingredient(ingredient, ingredientsdf):
    a = ingredientsdf.loc[ingredientsdf['ingredient'] == ingredient].recipe.to_list()
    return a
def ingredienten_bij_recept(recept, ingredientsdf):
    b = ingredientsdf.loc[ingredientsdf['recipe'] == recept].ingredient.unique().tolist()
    return b

#TAGS
# lijst met recepten of tags ophalen
def recepten_bij_tag(tag, ingredientsdf):
    a = tagsdf.loc[tagsdf['tag'] == tag].recipe.to_list()
    return a

def tags_bij_recept(gerecht, ingredientsdf):
    b = tagsdf.loc[tagsdf['recipe'] == gerecht].tag.unique()
    return b

### Meest voorkomende avondeten ingrediënten
Ik krimp de ingredients dataset naar alleen avondmaaltijdrecepten en selecteer de 4 meest voorkomende ingredienten.

In [4]:
#AVONDETEN
# alleen avondeten (tag 'diner' en 'hoofdgerecht')
diner_recepten = list(set(recepten_bij_tag('diner', ingredientsdf) + recepten_bij_tag('hoofdgerecht', ingredientsdf)))

# alle ingredienten en tags uit deze avondeten-recepten to list
list_of_ingredients = ingredientsdf[ingredientsdf['recipe'].isin(diner_recepten)].ingredient.unique()
list_of_Tags = tagsdf[tagsdf['recipe'].isin(diner_recepten)].tag.unique().tolist()

# tagsdf en ingredientdf krimpen naar avondeten
tagsdf = tagsdf[tagsdf['recipe'].isin(diner_recepten)]

ingreddf = ingredientsdf[ingredientsdf['recipe'].isin(diner_recepten)]

In [5]:
#MEEST VOORKOMENDE ITALIAANSE
italiaanse_recepten = list(recepten_bij_tag('italiaans', ingreddf))
#italiaanse_ingredients = ingredientsdf[ingredientsdf['recipe'].isin(italiaanse_recepten)].ingredient.unique() #1469
ingreddf_italiaans = ingredientsdf[ingredientsdf['recipe'].isin(italiaanse_recepten)]
ingreddf_italiaans = ingreddf_italiaans.groupby(['ingredient'])['ingredient'].count().to_frame()
ingreddf_italiaans = ingreddf_italiaans.rename(columns={'ingredient':'totaal'})
ingreddf_italiaans = ingreddf_italiaans.sort_values(['totaal'], ascending= False)

#MEEST VOORKOMENDE NEDERLANDSE
nederlandse_recepten = list(recepten_bij_tag('hollands', ingreddf))

ingreddf_NL = ingredientsdf[ingredientsdf['recipe'].isin(nederlandse_recepten)]
ingreddf_NL = ingreddf_NL.groupby(['ingredient'])['ingredient'].count().to_frame()
ingreddf_NL = ingreddf_NL.rename(columns={'ingredient':'totaal'})
ingreddf_NL = ingreddf_NL.sort_values(['totaal'], ascending= False)

In [6]:
ingreddf_NL

,totaal
ingredient,
ongezouten roomboter,121
traditionele olijfolie,92
knoflook,78
halfvolle melk,65
ui,64
...,...
kaas-uienbol,1
kaaskoekjes bladerdeeg,1
kabeljauw,1


In [7]:
# meest voorkomende 4 van beide selecteren
meeste_italiaans = ingreddf_italiaans.iloc[:4,0].to_frame().index.to_list()
meeste_NL = ingreddf_NL.iloc[:4,0].to_frame().index.to_list()

# uit te breiden bij groter voorbeeld

In [8]:
meeste_ingredienten_lijst = list(set(meeste_italiaans + meeste_NL))

In [9]:
meeste_ingredienten_lijst

['milde olijfolie',
 'knoflook',
 'halfvolle melk',
 'ui',
 'traditionele olijfolie',
 'ongezouten roomboter']

### Train en Test creëren
Random recepten uit nederlandse en italiaanse keuken zitten in de train en test set. je wil verzamelingen van random recepten in eerste helft tabel.

In [10]:
#FUNCTIE 10 RANDOM RECEPTEN
def random_recipes(list_of_recepten, K=10):
    random_recepten = random.choices(list_of_recepten, k=K)
    return random_recepten

#FUNCTIE VERZAMELINGEN RANDOMRECEPTEN
def verzamelingen_recipes(list_of_recepten, K=10, H=10):
    verzamelingen_recepten = []
    while len(verzamelingen_recepten) < H:
        verzamelingen_recepten.append(random_recipes(list_of_recepten, K=K))
    Train_verzamelingen = verzamelingen_recepten[:int(H*0.8)]
    Test_verzamelingen = verzamelingen_recepten[int(H*0.8):]
    return Train_verzamelingen, Test_verzamelingen

In [11]:
italiaanse_train, italiaanse_test = verzamelingen_recipes(italiaanse_recepten)

### Dummy data
nu worden de random recepten in dataframe gezet zodat deze omgezet kan worden naar matrix met 0en en 1en. de meest voorkomende ingrediënten als kolommen.

In [12]:
#INGREDIENTEN PER VERZAMELING FUNCTIE
def ingredienten_per_verzameling(set_verzamelingen):
    ingredienten_verzameling = []
    for j in set_verzamelingen:
        lijst_ingred = ingredienten_bij_recept(j, ingreddf)
        ingredienten_verzameling += lijst_ingred
    return ingredienten_verzameling

In [13]:
# dit is een lijst met lijsten waarin per verzameling alle ingredienten die voorkomen in recepten van die verzameling staan
train_ingred_verzamelingen = []
alle_verzamelingen = []
for set_verzamelingen in italiaanse_train:
    train_ingred_verzamelingen.append(ingredienten_per_verzameling(set_verzamelingen))
    alle_verzamelingen.append(set_verzamelingen)

In [14]:
# dit in dictionary zetten, zodat makkelijk naar df om te vormen
dict_train = {}
for i in range(0,len(alle_verzamelingen)):
    verzameling_ingred_value = train_ingred_verzamelingen[i]
    dict_train[i] = verzameling_ingred_value

In [15]:
# dataframe voor trainingsdata aanmaken van de dict
train_df1 = pd.DataFrame(index=dict_train, columns=meeste_ingredienten_lijst)
train_df1 = train_df1.fillna(0)

In [16]:
train_df1.columns # je ziet dat meest voorkomende ingredienten kolommen zijn

Index(['milde olijfolie', 'knoflook', 'halfvolle melk', 'ui',
       'traditionele olijfolie', 'ongezouten roomboter'],
      dtype='object')

In [17]:
# 0en en 1en plaatsen in df
for i in range(0,len(train_ingred_verzamelingen)):
    ingredienten_i = train_ingred_verzamelingen[i]
    overlap_lijsten = list(set(ingredienten_i) & set(meeste_ingredienten_lijst))
    if len(overlap_lijsten) != 0:
        for j in train_df1.columns:
            if j in overlap_lijsten:
                col_index = train_df1.columns.get_loc(str(j))
                train_df1.iloc[i,col_index] = 1

In [18]:
train_df1 # hoe t er nu uitziet

,milde olijfolie,knoflook,halfvolle melk,ui,traditionele olijfolie,ongezouten roomboter
0,1,1,1,1,1,1
1,1,1,0,1,1,0
2,1,1,0,0,1,1
3,1,1,0,0,1,1
4,0,0,0,1,1,0
5,1,1,0,1,1,1
6,1,1,1,1,1,1
7,1,1,1,1,1,1


In [19]:
df_leeg = pd.DataFrame()
train_concat = pd.concat([df_leeg, train_df1])

In [20]:
train_concat

,milde olijfolie,knoflook,halfvolle melk,ui,traditionele olijfolie,ongezouten roomboter
0,1,1,1,1,1,1
1,1,1,0,1,1,0
2,1,1,0,0,1,1
3,1,1,0,0,1,1
4,0,0,0,1,1,0
5,1,1,0,1,1,1
6,1,1,1,1,1,1
7,1,1,1,1,1,1


# Functie voor meerdere trains en daarna grotere matrix:

In [21]:
def create_train_enmatrix(lijst_recepten, K=10,H=10,R=3):
    df_train_links = pd.DataFrame()
    trainings = []
    while len(trainings) <= R:
        # aanmaken train (en test)
        Train, Test = verzamelingen_recipes(lijst_recepten,K=K,H=H)
        train_ingred_verzamelingen = []
        alle_verzamelingen = []
        for set_verzamelingen in Train:
            train_ingred_verzamelingen.append(ingredienten_per_verzameling(set_verzamelingen))
            alle_verzamelingen.append(set_verzamelingen)
        
        # dict: index tegenover ingredienten van verzameling
        dict_train = {}
        for i in range(0,len(alle_verzamelingen)):
            #verzameling_key = alle_verzamelingen[i]
            verzameling_ingred_value = train_ingred_verzamelingen[i]
            dict_train[i] = verzameling_ingred_value
        trainings.append(dict_train)
        
        #matrix vullen
        train_df = pd.DataFrame(index=dict_train, columns=meeste_ingredienten_lijst)
        train_df = train_df.fillna(0)
        
        #....df vullen
        for i in range(0,len(train_ingred_verzamelingen)):
            ingredienten_i = train_ingred_verzamelingen[i]
            overlap_lijsten = list(set(ingredienten_i) & set(meeste_ingredienten_lijst))
            if len(overlap_lijsten) != 0:
                for j in train_df.columns:
                    #print(train_df1.columns.get_loc(str(j)))
                    if j in overlap_lijsten:
                        col_index = train_df.columns.get_loc(str(j))
                        train_df.iloc[i,col_index] = 1
        
        #dfs aan elkaar plakken
        df_train_links = pd.concat([df_train_links, train_df])
    
    #index df fixen (van [0 : 8] naar [0 : 0.8*H*R])
    df_train_links = df_train_links.set_index(pd.Index(range(0, int(0.8*H*(R+1)))))
    
    return df_train_links

In [22]:
df = create_train_enmatrix(italiaanse_recepten)

In [23]:
df

,milde olijfolie,knoflook,halfvolle melk,ui,traditionele olijfolie,ongezouten roomboter
0,0,1,0,1,1,1
1,1,1,0,1,1,0
2,0,1,1,1,1,1
3,1,1,0,1,1,1
4,1,1,0,1,1,1
5,1,1,0,1,1,1
6,1,1,1,0,1,1
7,1,1,0,0,1,1
8,0,1,0,1,1,0
9,1,1,1,1,1,1


# Conclusie
Hierna zou nog de rechterhelft van de matrix gecreeërd moeten worden en een y kolom (met 1 voor zelfde keuken en 0 voor niet zelfde keuken) eraan geplakt worden om de trainset, geschikt voor een model, af te maken. Vervolgens wordt op soortgelijke manier een testset gecreëerd en kan er een classificatie model over getrained worden wat voorspeld of het uit dezelfde keuken komt of niet.